In [15]:
import requests
import pandas as pd
import sqlite3


COUNTRIES = ["TUR", "GBR", "USA"]
WORLD_BANK_URL_TEMPLATE = "https://api.worldbank.org/v2/country/{}/indicator/NY.GDP.MKTP.CD?format=json&date=1990:2023"


def fetch_world_bank_data(country):
    url = WORLD_BANK_URL_TEMPLATE.format(country)
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data[1]  
    else:
        print(f"{country} için API isteği başarısız oldu.")
        return []


df_list = []
for country in COUNTRIES:
    data = fetch_world_bank_data(country)
    df_country = pd.DataFrame([{ "country": country, "year": item["date"], "gdp": item["value"] } 
                               for item in data if item["value"] is not None])
    df_list.append(df_country)

df = pd.concat(df_list, ignore_index=True)


conn = sqlite3.connect("world_bank_data.db")
cursor = conn.cursor()


cursor.execute('''
    CREATE TABLE IF NOT EXISTS gdp_data (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        country TEXT,
        year INTEGER,
        gdp REAL
    )
''')


df.to_sql("gdp_data", conn, if_exists="replace", index=False)
conn.commit()
conn.close()


df.to_csv("world_bank_gdp.csv", index=False)
print("GSYİH verileri başarıyla SQL'e kaydedildi ve CSV olarak dışa aktarıldı.")


GSYİH verileri başarıyla SQL'e kaydedildi ve CSV olarak dışa aktarıldı.


In [19]:
import pandas as pd
import sqlite3


df = pd.read_csv("world_bank_gdp.csv")


conn = sqlite3.connect("world_bank_data.db")
cursor = conn.cursor()


cursor.execute('''
    CREATE TABLE IF NOT EXISTS gdp_data (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        country TEXT,
        year INTEGER,
        gdp REAL
    )
''')


df.to_sql("gdp_data", conn, if_exists="replace", index=False)


conn.commit()
conn.close()

print("CSV verisi başarıyla SQLite veritabanına aktarıldı!")


CSV verisi başarıyla SQLite veritabanına aktarıldı!


In [23]:
import sqlite3

conn = sqlite3.connect("world_bank_data.db")
cursor = conn.cursor()


cursor.execute("DROP TABLE IF EXISTS gdp_data")


cursor.execute('''
    CREATE TABLE gdp_data (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        country TEXT,
        year INTEGER,
        gdp REAL
    )
''')

conn.commit()
conn.close()
